<a href="https://colab.research.google.com/github/easycloudapi/learn_gcp/blob/main/learning_resources/05_GCP_CloudFunction_Service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview Of Cloud Function

Ref: https://cloud.google.com/functions/docs/concepts/overview

# Cloud Function (1st Gen)

# Cloud Function (2nd Gen)

**Cloud Functions (2nd gen)** is the next version of Google Cloud Functions, Google Cloud's Functions-as-a-Service offering.

This new version comes with an advanced feature set and is now powered by **Cloud Run** and **Eventarc** and **Cloud Build**, **Artifact Registry**

## Enhanced Infrastructure:

1. **Longer request processing:** Run your Cloud Functions longer than the 5 minute default, making it easier to run longer request workloads such as processing large streams of data from Cloud Storage or BigQuery. For HTTP functions, this is up to 60 minutes. For event-driven functions, this is currently up to 10 minutes.
2. **Larger instances:** Take advantage of up to 16GB of RAM and 4 vCPUs on Cloud Functions allowing larger in-memory, compute-intensive and more parallel workloads.
3. **Concurrency:** Process up to 1000 concurrent requests with a single function instance, minimizing cold starts and improving latency when scaling.
4. **Minimum instances:** Provide for pre-warmed instances to cut your cold starts and make sure the bootstrap time of your application does not impact the application performance.
5. **Traffic splitting:** Support multiple versions of your functions, split traffic between different versions and roll your function back to a prior version.

## Concurrency

Ref: https://codelabs.developers.google.com/codelabs/cloud-starting-cloudfunctions-v2?hl=en#8

**In Cloud Functions (2nd gen)**, a function instance handles 1 concurrent request by default but you can specify the number of concurrent requests that can be processed simultaneously by an instance. This can also be useful in preventing cold starts as a new function instance does not need to be created for every parallel request.

To fix the cold-start problem, you will deploy another function with a concurrency value of 100.

# Cloud FUnction 1st Gen Vs 2nd Gen

![image](https://github.com/easycloudapi/learn_gcp/assets/108976294/a1381b09-edcc-40ce-b58e-b21e70471d2e)

# Sample Pipeline 1: Pubsub to Bigquery Using Cloud Function

Ref: https://codelabs.developers.google.com/codelabs/cloud-starting-cloudfunctions-v2?hl=en#3

## Step 1: Setup and Requirements

```shell
gcloud config set project test-flask-api-372913

PROJECT_ID=$(gcloud config get-value project) \
REGION=us-west1
```

```shell
# enable the google services

gcloud services enable \
  artifactregistry.googleapis.com \
  cloudfunctions.googleapis.com \
  cloudbuild.googleapis.com \
  eventarc.googleapis.com \
  run.googleapis.com \
  logging.googleapis.com \
  pubsub.googleapis.com
```

```shell
# create pubsub topic
TOPIC=cloud-functions-gen2-topic

gcloud pubsub topics create $TOPIC
```

## Step 2: Create Cloud Function

1. Open Google Cloud Function UI
2. Create New 2nd Gen Function
```shell
gcloud functions deploy python-pubsub-function \
  --gen2 \
  --runtime python39 \
  --entry-point hello_pubsub \
  --source . \
  --region $REGION \
  --trigger-topic $TOPIC
```

### main.py

```python
import functions_framework

@functions_framework.cloud_event
def hello_pubsub(cloud_event):
   print('Pub/Sub with Python in GCF 2nd gen! Id: ' + cloud_event['id'])
```

### requirements.txt

```shell
functions-framework==3.*
```

## Step 3: Test the Cloud Function

```shell
# send msg to Pubsub Topic
gcloud pubsub topics publish $TOPIC --message="Hello World"

# check cloud logging, check same message id
gcloud functions logs read python-pubsub-function \
  --region $REGION --gen2 --format "value(log)"
```